In [1]:
import os
import numpy as np
import pandas as pd
import datetime
from pathlib import Path
from pprint import pprint
import configparser
import ezomero
from ezomero import rois
#   https://thejacksonlaboratory.github.io/ezomero/ezomero.html#module-ezomero
#   https://docs.openmicroscopy.org/ome-model/5.6.3/developers/roi.html
#   https://github.com/HdBraanker/Spectral_Flow_Workflow/blob/main/script.Rmd
datestr = datetime.datetime.now().strftime('%Y_%m_%d')
print(datestr)

2024_10_08


In [2]:
# Settings for OMERO
home = str(Path.home())
config = configparser.RawConfigParser()
config.read(r'{}\Documents\myKeyVariables.cfg'.format(home))   
envVars = dict(config.items('OMERO'))

# Settings for OMERO
user = envVars['user']
password = envVars['pass']
host = envVars['url']
port = 4064 #4080
secure = True #False
outDir = envVars['pointsdownloaded']
lab = envVars['labname']

conn = ezomero.connect(user, password, lab, host=host, port=port, secure=secure)
ezomero.print_projects(conn)

Projects:
	BMS_Teir2_2022:	102
	BMS_Tier1_2021:	201
	3D Analysis MxIF:	915
	IHC_v_MxIF_Developments:	306
	MelPanel4:	311
	BMS_CoreBiopsies_2023:	405
	IHC_v_MxIF_For_March2023_PathReview:	475
	Tassos_updated_2023:	457
	CosMx to CellDive Alignments:	606
	MISCELLANEOUS:	652
	IHC_v_MxIF_June6_PathReview:	712
	Elizabeth's Placenta Project:	751
	Lung Project :	752
	GM_CSF_Pilot:	807
	IHC_v_MxIF_Sept2023_PathReview:	713
	IHCs for CD36 and FASN:	865
	Mackenzie Study and RO1 Practice:	857
	20230620_IHC_MxIF_Dev_CD19_MPO_NRP2:	853
	BMS_Aim2_Pratice:	1005
	BMS_Aim2_Batch1:	1264
	BMS_Aim2_Batch1_Gastric:	1269
	BMS_2024_Mel_GI_H&E:	1274
	Pregnancy_Cancer:	1276


In [3]:
ezomero.print_datasets(conn, project=1264)
allDatasets = ezomero.get_dataset_ids(conn, project=1264)

Datasets in Project "BMS_Aim2_Batch1":
	BAD QUALITY:	2115
	Melanoma:	2048


In [4]:
## Image Desc
## https://docs.openmicroscopy.org/omero/5.4.1/developers/PythonBlitzGateway.html
# https://downloads.openmicroscopy.org/omero/5.4.10/api/python/omero/omero.gateway.html#omero.gateway._ImageWrapper
fout = open(os.path.join(outDir,'log_omero_{}.txt'.format(datestr) ), 'w')
for sm in allDatasets:
    directory_path = os.path.join(outDir,"Dataset_"+str(sm))
    # Check if the directory exists
    if not os.path.exists(directory_path):
        # Create the directory
        os.makedirs(directory_path)
        print(f"Directory '{directory_path}' created successfully.")
    
    imgs = ezomero.get_image_ids(conn, dataset=sm)
    for im in imgs:
        imgDesc, pix = ezomero.get_image(conn, im, xyzct=False, no_pixels=True)
        if(imgDesc.getROICount(shapeType="Point") > 0):
            fout.write(str(sm)+"  :  "+imgDesc.getName()+'\n')
            shName = imgDesc.getName().replace(".ome.tiff",'')
            imgPointsIDs = ezomero.get_roi_ids(conn, im)
            print("{} has {}".format(imgDesc.getName().replace('\\.ome\\.tiff',''), len(imgPointsIDs)))
            fout.write("{} Annotations\n".format(len(imgPointsIDs)))
            allThesePoints = []
            for i in imgPointsIDs:
                for ppI in ezomero.get_shape_ids(conn, i):
                    try:
                        pt, fill, border, width = ezomero.get_shape(conn, ppI)
                        allThesePoints.append([pt.x, pt.y, pt.label,pt.label, fill, border, width])
                    except:
                        print("   Not point ID : "+str(i))
            #pprint(allThesePoints)
            pointTable = pd.DataFrame(allThesePoints)
            pointTable.columns = ["x","y","class", "name","fill","border","width"]
            # Replace values in "Classification" column
            pointTable['class'] = pointTable['class'].replace({
                'B cell': 'B Cell',
                'T reg': 'T Reg'
            })
            
            cntTbl = pointTable['class'].value_counts().to_frame()
            noms = cntTbl.index.to_list()
            cnts = cntTbl.values
            for n in range(len(noms)):
                fout.write("{}\t{}\n".format(noms[n],cnts[n][0]))
            fout.write("\n")
            pointTable.to_csv(os.path.join(directory_path,shName+'-points.tsv'), index=False, sep="\t")


Directory 'Y:\AnalysisPipelineProjects\BMS2_Melanoma\OMERO_POINTS\Dataset_2048' created successfully.
SLIDE-3001_FullPanel.ome.tiff has 152
   Not point ID : 76796
SLIDE-3016_FullPanel.ome.tiff has 437
   Not point ID : 76752
   Not point ID : 76753
   Not point ID : 76754
   Not point ID : 76755
   Not point ID : 76756
   Not point ID : 76757
   Not point ID : 76758
   Not point ID : 76759
   Not point ID : 76760
   Not point ID : 76761
   Not point ID : 76762
   Not point ID : 76763
   Not point ID : 76764
   Not point ID : 76765
SLIDE-3019_FullPanel.ome.tiff has 13
   Not point ID : 76749
   Not point ID : 77010
   Not point ID : 77011
SLIDE-3020_FullPanel.ome.tiff has 451
SLIDE-3021_FullPanel.ome.tiff has 470
Directory 'Y:\AnalysisPipelineProjects\BMS2_Melanoma\OMERO_POINTS\Dataset_2115' created successfully.
SLIDE-3001_FullPanel.ome.tiff has 152
   Not point ID : 76796


In [5]:
fout.close()